# Linear Regression Problem Code Tutorial

<b><u>[목적]</u></b>
- 단순 Linear Regression을 활용하여 변수의 중요도 및 방향성을 알아봄
- 매우 심플한 모델이기 때문에 사이즈가 큰 데이터에 적합하지 않음
- 하지만 설명력에서는 큰 장점이 있음

<b><u>[Process]</u></b>
- Data Path = 'https://github.com/GonieAhn/Data-Science-online-course-from-gonie/tree/main/Data%20Store'
- Define X's & Y
- Split Train & Valid data set
- Modeling
- 해석

<b><u>[고민 Points]</u></b>
- scaling이 필요한가?

In [1]:
import os
import gc
import pickle
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from regressors import stats

In [2]:
%%time
# Data Load 
data = pd.read_csv("../Data Store/TOY_DATA.csv")
print(">>>> Data Shape : {}".format(data.shape))

>>>> Data Shape : (3500, 357)
Wall time: 232 ms


In [3]:
# Drop Unnamed:0 Columns
data.drop(columns=['Unnamed: 0'], inplace=True)
data.reset_index(inplace=True, drop=True)
print(">>>> Data Shape : {}".format(data.shape))

>>>> Data Shape : (3500, 356)


<b><u>[Data Selection]</u></b>
- Linear Regression의 경우 심플한 모델이기 때문에 변수를 Domain을 바탕으로 변수를 추출하여 분석을 진행함

In [4]:
# Feature Selection
selc_col = ['Y', 'X1', 'X2', 'X3']
data = data[selc_col]
# Missing value dropping
data.dropna(inplace=True)
data.reset_index(inplace=True, drop=True)
print("Data Shape : {}".format(data.shape))

Data Shape : (3500, 4)


In [5]:
# Define X's and Y
Y = data['Y']
X = data.drop(columns=['Y']) 

<b><u>[Data Split]</u></b>
- Data Split을 진행할 때 BigData의 경우 꼭 indexing을 추출하여 모델에 적용시켜야 함
- 이유는 Data Split하여 새로운 Data set을 만들 경우 메모리에 부담을 주기 때문

In [6]:
# Indexing Split for memory management
idx = list(range(X.shape[0]))
train_idx, valid_idx = train_test_split(idx, test_size=0.3, random_state=2021)
print(">>>> # of Train data : {}".format(len(train_idx)))
print(">>>> # of valid data : {}".format(len(valid_idx)))

>>>> # of Train data : 2450
>>>> # of valid data : 1050


In [7]:
# Data 설명
data.describe()

,Y,X1,X2,X3
count,3500.000000,3500.000000,3500.000000,3500.000000
mean,0.765216,84.757693,11.058408,0.705708
std,0.103392,0.351666,0.040699,0.002689
min,0.000000,81.661900,10.791300,0.684082
25%,0.712546,84.561100,11.032575,0.704376
50%,0.777598,84.783400,11.060500,0.705982
75%,0.834474,85.001150,11.086100,0.707514
max,1.000000,85.745600,11.161800,0.712631


<b><u>[X's Scaling(X) & 결과 해석]</u></b>
- Validation Data Set에 대해 R-squared가 0.99이기 때문에 잘 구축된 모델임 (아래 Cell의 표)
- X1의 Estimate(Coefficient)가 0.175임 X1를 1단위 증가시켰을 때 Y가 0.175 증가한다는 의미임
- X1, X2, X3의 Estimate(Coefficient)의 단위가 다른이유는 변수들간 Scale이 다르기 때문임
- 따라서 변수 랭킹을 따지기는 어려움

In [8]:
# Linear Regression
results = LinearRegression().fit(X.iloc[train_idx], Y.iloc[train_idx])
stats.summary(results, X.iloc[valid_idx], Y.iloc[valid_idx], xlabels=list(X.columns)) 

Residuals:
    Min      1Q  Median     3Q    Max
-0.0082 -0.0002     0.0 0.0002 0.0024


Coefficients:
             Estimate  Std. Error     t value  p value
_intercept -43.786227    0.008074  -5422.7829      0.0
X1           0.175347    0.000005  38893.8624      0.0
X2           1.343589    0.000262   5118.6526      0.0
X3          21.016459    0.007953   2642.7444      0.0
---
R-squared:  0.99997,    Adjusted R-squared:  0.99997
F-statistic: 13365715.16 on 3 features


In [9]:
# RMSE 계산
pred_y = results.predict(X.iloc[valid_idx])
RMSE = np.sqrt(np.mean((pred_y - Y.iloc[valid_idx])**2))
print(RMSE)

0.0005595926646214808


<b><n>[X's Scaling(O) & 결과 해석]</n></b>
- Validation Data Set에 대해 R-squared가 0.99이기 때문에 잘 구축된 모델임 (아래 Cell의 표)
- X1의 Estimate(Coefficient)가 0.569임
    - X1를 1단위 증가시켰을 때 Y가 0.569 증가한다는 의미라고 판단할 수 없음
    - 이유는 Scaling을 진행 했기 때문임
- 하지만 변수들간 중요도 랭킹은 세울수 있음 (Scaling을 했기 때문에)
    - Top1 - X3
    - Top2 - X1
    - Top3 - X2

In [10]:
# Scaling
scaler = MinMaxScaler().fit(X.iloc[train_idx])
X_scal = scaler.transform(X)
X_scal = pd.DataFrame(X_scal, columns=X.columns)

In [11]:
# Linear Regression
results = LinearRegression().fit(X_scal.iloc[train_idx], Y.iloc[train_idx])
stats.summary(results, X_scal.iloc[valid_idx], Y.iloc[valid_idx], xlabels=list(X_scal.columns))

Residuals:
    Min      1Q  Median     3Q    Max
-0.0082 -0.0002     0.0 0.0002 0.0024


Coefficients:
            Estimate  Std. Error    t value  p value
_intercept -0.444687    0.000162 -2750.3142      0.0
X1          0.569737    0.000165  3461.5895      0.0
X2          0.497800    0.000174  2868.7659      0.0
X3          0.599999    0.000199  3014.6263      0.0
---
R-squared:  0.99997,    Adjusted R-squared:  0.99997
F-statistic: 13365715.16 on 3 features


In [12]:
# RMSE 계산
pred_y = results.predict(X_scal.iloc[valid_idx])
RMSE = np.sqrt(np.mean((pred_y - Y.iloc[valid_idx])**2))
print(RMSE)

0.0005595926646210362


<b><n>[Regressors package install error]</n></b>
- Regressors의 Package는 conda에서 지원하지 않음
- https://pypi.org/project/regressors/#files
    - .gz 파일을 다운로드 함
- anaconda prom 키고 cd를 통해 .gz 파일 다운로드한 폴더로 이동
    - tar xvzf regressors.x.x.x.tar.gz 명령어를 통하여 압축 해제
    - cd regressors.x.x.x 명령어를 통하여 압축 해제한 파일로 들어감
    - python setup.py install 명령어로 직접 설치해줌
        - 주의: 여기서 "use_2to3 is invalid" 라는 에러 뜨면 아래와 같이 해결
            - setup.py 파일을 메모장으로 열기
            - "use_2to3=True" --> "use_2to3=False"로 변경한 후 저장
            - 다시 설치 하면 됨